In [1]:
%%capture
%pip install iblm --upgrade

In [2]:
%pip list | grep iblm

iblm                         0.3.55

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np

import iblm
from sklearn.datasets import make_classification

In [4]:
num_train_data = 10000 # 50〜300
num_test_data = 1000
sample = num_train_data + num_test_data
n_informative = 2
n_redundant = 0
n_features = n_informative + n_redundant
weights = [0.5, 0.5]
flip_y=0
seed = 3655  # 3655,3656,3657

# testデータの個数を揃えるためにtrain_test_splitを使っていない
x, y = make_classification(
    n_samples = sample,  # データ数
    n_features = n_features,  # 特徴量の数
    n_informative = n_informative,  # ラベル予測に意味のある特徴量の数
    n_redundant = n_redundant,  # 冗長な特徴量
    weights = weights,  # [0,1]の割合
    flip_y = flip_y, # 逆のラベルに反転する割合
    random_state = seed
    )

x = np.round(x, decimals=3)
x = pd.DataFrame(x)
y = y.astype(int)

x_test = x[0:num_test_data]
x_train = x[num_test_data:]

y_test = y[:num_test_data]
y_train = y[num_test_data:]

In [5]:
iblbg = iblm.IBLBaggingModel(
    api_type="openai",
    #api_type="gemini",
    model_name="gpt-4-0125-preview",
    #model_name="gemini-pro",
    objective="binary",
    num_model=20,
    max_sample = 2000,
    min_sample = 300,
    )

In [6]:
code_models = iblbg.fit(
    x_train,
    y_train,
    temperature=0.3
    )

model_1


2024-03-10 15:24:23,117 [iblm.ibl][INFO] (ibl:ibl.py:fit:150)


model_2


2024-03-10 15:24:28,876 [iblm.ibl][INFO] (ibl:ibl.py:fit:150)


model_3
Error: ['Traceback (most recent call last):\n', '  File "/Users/tanakatouma/vscode/IBLM/.venv/lib/python3.9/site-packages/iblm/iblbagging/iblbagging.py", line 82, in fit\n    bagging_model = super().fit(\n', '  File "/Users/tanakatouma/vscode/IBLM/.venv/lib/python3.9/site-packages/iblm/ibl.py", line 142, in fit\n    code_model = self._run_prompt(prompt=prompt_, seed=seed, temperature=temperature)\n', '  File "/Users/tanakatouma/vscode/IBLM/.venv/lib/python3.9/site-packages/iblm/ibl.py", line 109, in _run_prompt\n    return run_prompt(\n', '  File "/Users/tanakatouma/vscode/IBLM/.venv/lib/python3.9/site-packages/iblm/llm_client.py", line 84, in run_prompt\n    response = model.generate_content(prompt)\n', '  File "/Users/tanakatouma/vscode/IBLM/.venv/lib/python3.9/site-packages/google/generativeai/generative_models.py", line 248, in generate_content\n    response = self._client.generate_content(request)\n', '  File "/Users/tanakatouma/vscode/IBLM/.venv/lib/python3.9/site-package

...

2024-03-10 15:27:16,796 [iblm.ibl][INFO] (ibl:ibl.py:fit:150)


model_19


2024-03-10 15:27:50,230 [iblm.ibl][INFO] (ibl:ibl.py:fit:150)


Error: ['Traceback (most recent call last):\n', '  File "/Users/tanakatouma/vscode/IBLM/.venv/lib/python3.9/site-packages/iblm/iblbagging/iblbagging.py", line 93, in fit\n    metric_dict = super().evaluate(y_sampled, y_pred)\n', '  File "/Users/tanakatouma/vscode/IBLM/.venv/lib/python3.9/site-packages/iblm/ibl.py", line 187, in evaluate\n    return evaluate(y_true, y_pred, self.objective)\n', '  File "/Users/tanakatouma/vscode/IBLM/.venv/lib/python3.9/site-packages/iblm/metrics.py", line 24, in evaluate\n    y_pred = np.where(y_prob > 0.5, 1, 0)\n', "TypeError: '>' not supported between instances of 'NoneType' and 'float'\n"]
model_20


2024-03-10 15:27:59,232 [iblm.ibl][INFO] (ibl:ibl.py:fit:150)


In [7]:
code_models

[('model_9',
  {'code_model': '#########\ndef predict(x):\n    import numpy as np\n\n    df = x.copy()\n    output = []\n    for index, row in df.iterrows():\n        # Do not change the code before this point.\n        # Please describe the process required to make the prediction below.\n        if row[1] > 0:\n            y = 1\n        else:\n            y = 0\n\n\n        # Do not change the code after this point.\n        output.append(y)\n    return np.array(output)\n#########',
   'metric_dict': {'roc_auc': 0.8320886171119243,
    'pr_auc': 0.7657358915600379,
    'accuracy': 0.831858407079646,
    'recall': 0.8754448398576512,
    'precision': 0.803921568627451,
    'f1_score': 0.838160136286201}}),
 ('model_7',
  {'code_model': '#########\ndef predict(x):\n    import numpy as np\n\n    df = x.copy()\n    output = []\n    for index, row in df.iterrows():\n        # Do not change the code before this point.\n        # Please describe the process required to make the prediction b

In [8]:
top_model = 20

for i in range(1, top_model):
    y_pred = iblbg.predict_(x_test, i)
    print(iblbg.evaluate(y_test, y_pred))

{'roc_auc': 0.8727509000404468, 'pr_auc': 0.8320676982591877, 'accuracy': 0.873, 'recall': 0.8800773694390716, 'precision': 0.875, 'f1_score': 0.8775313404050145}
{'roc_auc': 0.8831789548718318, 'pr_auc': 0.8129537127180217, 'accuracy': 0.873, 'recall': 0.8800773694390716, 'precision': 0.875, 'f1_score': 0.8775313404050145}
{'roc_auc': 0.8797870338110855, 'pr_auc': 0.8104275738477311, 'accuracy': 0.845, 'recall': 0.8626692456479691, 'precision': 0.8415094339622642, 'f1_score': 0.8519579751671442}
{'roc_auc': 0.8705763863025657, 'pr_auc': 0.8032433311803204, 'accuracy': 0.823, 'recall': 0.839458413926499, 'precision': 0.821969696969697, 'f1_score': 0.830622009569378}
{'roc_auc': 0.8630837247858524, 'pr_auc': 0.7972474384351246, 'accuracy': 0.804, 'recall': 0.8220502901353965, 'precision': 0.8034026465028355, 'f1_score': 0.8126195028680688}
{'roc_auc': 0.8633139909735655, 'pr_auc': 0.7976006967195712, 'accuracy': 0.804, 'recall': 0.8220502901353965, 'precision': 0.8034026465028355, 'f1_s